# Take 2: Adaboost with Weak Classifiers
None of that bagging classifier bullshit

In [2]:
# Import guys
import pandas as pd
import numpy as np
from scipy import sparse
# from AdaBoostClassifier import AdaBoostClassifier
#%run AdaBoostWeak.py
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt


### Import data

In [3]:
# column names
with open('data/column_names.txt', 'r') as f:
    column_names = [line.strip() for line in f]

sparse_dat = sparse.load_npz("data/sparse_df.npz")

# Extract labels from the first column
labels = sparse_dat[:, 0]

# Create a list of column indices to keep
to_keep = list(set(range(sparse_dat.shape[1])) - set([0]))

# Extract the design matrix
X = sparse_dat[:, to_keep]

print(f'labels: \n{labels}')

UnicodeDecodeError: 'utf-8' codec can't decode byte 0xf3 in position 5587: invalid continuation byte

In [ ]:
print(f'Design matrix: \n{X}')

Design matrix: 
  (17173, 0)	1
  (17240, 0)	2
  (17164, 1)	1
  (17363, 1)	1
  (17448, 1)	1
  (17910, 1)	1
  (17914, 1)	1
  (17933, 1)	2
  (15801, 2)	1
  (15867, 2)	1
  (16217, 2)	1
  (17173, 2)	3
  (17189, 2)	1
  (17386, 2)	1
  (17765, 2)	4
  (17933, 3)	2
  (17325, 4)	1
  (17366, 4)	1
  (16271, 5)	1
  (16837, 5)	1
  (17933, 6)	2
  (17933, 7)	2
  (17173, 8)	2
  (17933, 8)	1
  (15801, 9)	1
  :	:
  (8386, 56199)	2
  (8669, 56200)	1
  (10316, 56200)	1
  (8805, 56201)	1
  (9218, 56201)	1
  (9565, 56202)	1
  (9814, 56202)	1
  (6232, 56203)	1
  (9353, 56203)	1
  (5661, 56204)	9
  (6860, 56204)	1
  (8698, 56204)	1
  (125, 56205)	1
  (1228, 56205)	1
  (4422, 56205)	1
  (5648, 56206)	1
  (7464, 56206)	1
  (5648, 56207)	2
  (6025, 56208)	2
  (6025, 56209)	2
  (6151, 56210)	2
  (8669, 56211)	1
  (10316, 56211)	1
  (6340, 56212)	1
  (8696, 56212)	1


### Train and Test split
To do: consider stratifying by type

In [ ]:
# To do - stratify the split 
n_samples = labels.shape[0]
# Use train_test_split.
X_train, X_test, y_train, y_test = train_test_split(X, labels, test_size=0.2, shuffle=False)

print(X_train.shape)
print(y_train.shape)
print(X_test.shape)
print(y_test.shape)
print(y_train.dtype)
print("proportion of spam in training data:", (y_train == 1).sum().item() / y_train.shape[0])
print("proportion of spam in testing data:", (y_test == 1).sum().item() / y_test.shape[0])

(14524, 56213)
(14524, 1)
(3632, 56213)
(3632, 1)
int64
proportion of spam in training data: 0.39568989259157256
proportion of spam in testing data: 0.11921806167400881


In [ ]:
def errors(y, y_pred):
        """
        Calculate the proportion of type 2 errors - when the true label is 1 - spam, and the predicted label is 0 - ham

        Args:
        y: true labels
        y_pred: predicted labels
        """
        n = y.shape[0]
        type2errors = ((y == 1) & (y_pred == 0)).sum().item()
        type1errors = ((y == 0) & (y_pred == 1)).sum().item()
        correct = (y_pred == y).sum().item()
        return type2errors, type1errors, correct

### Train/Test models

2 models:
1. Without penalty
2. With penalty

In [ ]:
%run AdaBoostWeak.py

aboost1 = AdaBoostWeak(type2penalty = False, rounds = 200, maxDTdepth = 5)
aboost1.fit(X = X_train, y = y_train.toarray().ravel())



In [ ]:
predictions = aboost1.predict(X_test)
type2, type1, correct = errors(y_test.toarray().ravel(), predictions)
print("Model 1 Test Accuracy: ", correct/len(predictions))
print(f'unique predictions - should be 0 and 1: {np.unique(predictions)}')
print(f'type 2 errors: {type2} \n type 1 errors: {type1}')
print(aboost1.alphas)
print(predictions)

Model 1 Test Accuracy:  0.5922356828193832
unique predictions - should be 0 and 1: [0 1]
type 2 errors: 8 
 type 1 errors: 1473
[1.9311982474293699, 1.1922207924007637, 0.6385318941643855, 0.9564294022214502, 0.816069472339187, 0.6922920016770775, 0.6369655768031284, 0.5580145171983981, 0.53676580386847, 0.4237535232743738, 0.4623203222935166, 0.4723842325214209, 0.4895676529238302, 0.3666700342477265, 0.35037721839934366, 0.5165341888678217, 0.5573056346856897, 0.4064054111284078, 0.34746557733007816, 0.3810543285918116, 0.2819655300587526, 0.3205254061893938, 0.2939073708421175, 0.30012073061184613, 0.364602270904182, 0.4133055679593059, 0.32306578039480827, 0.2772096917590599, 0.2989313921385927, 0.29654207372135993, 0.26341570358993205, 0.27857891332401874, 0.27555848356013546, 0.36463992483772384, 0.29851639244450395, 0.20130117787237253, 0.2509984065035767, 0.24513824181768812, 0.20405140928819254, 0.2110796620117299, 0.20614038627549833, 0.20741264415432778, 0.18315574030081108,

In [ ]:
predictions = aboost1.predict(X_train)
type2, type1, correct = errors(y_train.toarray().ravel(), predictions)
print("Model 1 (no penalty) Training set Accuracy: ", correct/len(predictions))
print(f'unique predictions - should be 0 and 1: {np.unique(predictions)}')
print(f'type 2 errors: {type2} \n type 1 errors: {type1}')

Model 1 (no penalty) Training set Accuracy:  0.9809281189754888
unique predictions - should be 0 and 1: [0 1]
type 2 errors: 140 
 type 1 errors: 137


### Model tuning/Hyperparameter search

Sigh. So much overfitting

In [ ]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import cross_val_score


clf = DecisionTreeClassifier(max_depth=9)
cvs = cross_val_score(clf, X_train,predictions,cv=4)
print("cross val scores ",cvs)


cross val scores  [0.82181217 0.98430185 0.91297163 0.86312311]


In [ ]:
# #checkity check
# from importlib import reload
# import AdaBoostWeak
# reload(AdaBoostWeak)
# from AdaBoostWeak import AdaBoostWeak
# a= AdaBoostWeak(rounds=100)
# a.get_params().keys()

dict_keys(['rounds', 'type2penalty', 'maxDTdepth'])

In [ ]:
# #clonity checkity check
# from sklearn.base import clone
# ada = AdaBoostWeak(rounds=100, type2penalty=False, maxDTdepth=1)  # Replace with actual parameters
# ada_clone = clone(ada)

In [ ]:
# #not adaboost classifier woops
# from sklearn.model_selection import GridSearchCV

# clf = DecisionTreeClassifier()
# param_grid = [
#     {'type2penalty': [False] ,'rounds': [200, 300, 400, 500], 'maxDTdepth': [5,6,7,8,9,10]},
#     {'type2penalty': [True] ,'rounds': [200, 300, 400, 500], 'maxDTdepth': [5,6,7,8,9,10]}
#     ]

# print(type(y_train))
# print(y_train.shape)
# y_train_dense = y_train.toarray().ravel()
# print(y_train.shape)

# grid_search = GridSearchCV(clf, param_grid, cv=4)
# grid_search.fit(X_train, y_train_dense)

# print(grid_search.best_params_)


aboost = AdaBoostWeak(rounds=100)
param_grid = [
    {'type2penalty': [False], 'rounds': [200, 300, 400, 500], 'maxDTdepth': [5, 6, 7, 8, 9, 10]},
    {'type2penalty': [True], 'rounds': [200, 300, 400, 500], 'maxDTdepth': [5, 6, 7, 8, 9, 10]}
]

print(type(y_train))
print(y_train.shape)

y_train_dense = y_train.toarray().ravel()

print(y_train_dense.shape)
grid_search = GridSearchCV(aboost, param_grid, cv=4)
grid_search.fit(X_train, y_train_dense)

print("best params:", grid_search.best_params_)


NameError: name 'AdaBoostWeak' is not defined

In [ ]:
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import make_scorer, accuracy_score
param_grid = [
    {'type2penalty': [False] ,'rounds': [200, 300, 400, 500], 'maxDTdepth': [5,6,7,8,9,10]},
    {'type2penalty': [True] ,'rounds': [200, 300, 400, 500], 'maxDTdepth': [5,6,7,8,9,10]}
    ]


model = AdaBoostWeak(type2penalty=False, maxDTdepth=5,rounds=100)
scorer = make_scorer(accuracy_score)
grid_search=GridSearchCV(estimator=model, param_grid=param_grid, scoring=scorer, cv=5)
grid_search.fit(X_train, y_train.toarray().ravel())

print("best params:", grid_search.best_params_)
print("best cross val score: {:.2f}".format(grid_search.best_score_))